# Исследование надёжности заёмщиков

Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.

Результаты исследования будут учтены при построении модели **кредитного скоринга** — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

## Шаг 1. Откройте файл с данными и изучите общую информацию

Подключим необходимые библиотеки для работы с данными.

In [1]:
import pandas as pd
import numpy as np

# pymystem3 импортируется так:
from pymystem3 import Mystem
m = Mystem() 

from collections import Counter

Загрузим датасет. Предусмотрим два варианта размещения данных: в локальной папке и на сервере. 

In [2]:
try:
    data = pd.read_csv('data.csv')
except:
    data = pd.read_csv('/datasets/data.csv')

Проверим данные на читаемость. Выведем на экран несколько строк.

In [3]:
data.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу


Данные читаемы, проблем с кодировкой, разделителями и тп нет, продолжим изучение.

In [4]:
data.columns

Index(['children', 'days_employed', 'dob_years', 'education', 'education_id',
       'family_status', 'family_status_id', 'gender', 'income_type', 'debt',
       'total_income', 'purpose'],
      dtype='object')

Наименования колонок заданы верно, пробелов в конце названий нет, переименование не требуется.

In [5]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


Итак, в таблице 12 столбцов. Типы данных: float64(2), int64(5), object(5).

Количество значений в некоторых столбцах меньше, чем строк в таблице: days_employed (19351), total_income (19351).
Значит, в данных есть пропущенные значения.

Проверим на соответствие здравому смыслу значения в каждом столбце.

Согласно документации к данным:

children — количество детей в семье

days_employed — общий трудовой стаж в днях

dob_years — возраст клиента в годах

education — уровень образования клиента

education_id — идентификатор уровня образования

family_status — семейное положение

family_status_id — идентификатор семейного положения

gender — пол клиента

income_type — тип занятости

debt — имел ли задолженность по возврату кредитов

total_income — ежемесячный доход

purpose — цель получения кредита

Начнем со столбцов с численными значениями.

In [6]:
data.describe()

,children,days_employed,dob_years,education_id,family_status_id,debt,total_income
count,21525.000000,19351.000000,21525.000000,21525.000000,21525.000000,21525.000000,1.935100e+04
mean,0.538908,63046.497661,43.293380,0.817236,0.972544,0.080883,1.674223e+05
std,1.381587,140827.311974,12.574584,0.548138,1.420324,0.272661,1.029716e+05
min,-1.000000,-18388.949901,0.000000,0.000000,0.000000,0.000000,2.066726e+04
25%,0.000000,-2747.423625,33.000000,1.000000,0.000000,0.000000,1.030532e+05
50%,0.000000,-1203.369529,42.000000,1.000000,0.000000,0.000000,1.450179e+05
75%,1.000000,-291.095954,53.000000,1.000000,1.000000,0.000000,2.034351e+05
max,20.000000,401755.400475,75.000000,4.000000,4.000000,1.000000,2.265604e+06


children - количество детей в семье: значения от -1 до 20, среднее 0,54

Минус 1 ребенок быть не может, надо проверить сколько таких значений. Остальные значения похожи на правду, максимальные 20 детей теоретически возможны (допустим семья с приемными детьми и тп), но также надо проверить сколько таких значений.

days_employed - общий трудовой стаж в днях: значения от -18 000 до 400 000, среднее 63 000.

Сразу видны проблемы: более 75% значений отрицательные (что не вяжется с описанием графы), очень большие значения (средние 63 000 дней это больше 172 лет).

dob_years - возраст клиента в годах

Смущает только минимальное значение "0", надо проверить сколько таких значений. Остальные значения выглядят правдоподобно.

education_id — идентификатор уровня образования

Целочисленные значения от 0 до 4 соответствуют уровням образования - правдоподобно.

family_status_id — идентификатор семейного положения

Целочисленные значения от 0 до 4 - правдоподобно.

debt — имел ли задолженность по возврату кредитов

Целочисленные значения 0 или 1 - приблизительно 8% имели задолженности - правдоподобно.

total_income — ежемесячный доход

Значения от 20 тысяч до 2 миллионов, медиана 145 тысяч, среднее 167 тысяч. Немного смущают среднее и медиана: это либо не Россия, либо не рубли, либо не то и не другое вместе. Дробный формат чисел может объясняться либо получением месячного дохода путем деления годового на 12, либо приведением к нужной валюте. 

Рассмотрим столбцы с нечисловыми значениями.

In [7]:
data.describe(include=[object])  

,education,family_status,gender,income_type,purpose
count,21525,21525,21525,21525,21525
unique,15,5,3,8,38
top,среднее,женат / замужем,F,сотрудник,свадьба
freq,13750,12380,14236,11119,797


In [8]:
# посмотрим какие значения имеются в графе 'education' и их количество
data['education'].value_counts()

среднее                13750
высшее                  4718
СРЕДНЕЕ                  772
Среднее                  711
неоконченное высшее      668
ВЫСШЕЕ                   274
Высшее                   268
начальное                250
Неоконченное высшее       47
НЕОКОНЧЕННОЕ ВЫСШЕЕ       29
НАЧАЛЬНОЕ                 17
Начальное                 15
ученая степень             4
Ученая степень             1
УЧЕНАЯ СТЕПЕНЬ             1
Name: education, dtype: int64

education — уровень образования клиента

15 уникальных значений, имеются дубликаты (идентификаторов этой категории всего 5)

In [9]:
# посмотрим какие значения имеются в графе 'family_status' и их количество
data['family_status'].value_counts()

женат / замужем          12380
гражданский брак          4177
Не женат / не замужем     2813
в разводе                 1195
вдовец / вдова             960
Name: family_status, dtype: int64

family_status — семейное положение

5 уникальных значений (идентификаторов этой категории также 5), всё ок.

In [10]:
# посмотрим какие значения имеются в графе 'gender' и их количество
data['gender'].value_counts()

F      14236
M       7288
XNA        1
Name: gender, dtype: int64

gender — пол клиента

3 уникальных значения, третий вариант "XNA" встречается всего 1 раз, надо будет заменить.

In [11]:
# посмотрим какие значения имеются в графе 'income_type' и их количество
data['income_type'].value_counts()

сотрудник          11119
компаньон           5085
пенсионер           3856
госслужащий         1459
безработный            2
предприниматель        2
студент                1
в декрете              1
Name: income_type, dtype: int64

income_type — тип занятости

8 уникальных значений, правдоподобно.

In [12]:
# посмотрим какие значения имеются в графе 'purpose' и их количество
data['purpose'].value_counts()

свадьба                                   797
на проведение свадьбы                     777
сыграть свадьбу                           774
операции с недвижимостью                  676
покупка коммерческой недвижимости         664
покупка жилья для сдачи                   653
операции с жильем                         653
операции с коммерческой недвижимостью     651
покупка жилья                             647
жилье                                     647
покупка жилья для семьи                   641
строительство собственной недвижимости    635
недвижимость                              634
операции со своей недвижимостью           630
строительство жилой недвижимости          626
покупка недвижимости                      624
строительство недвижимости                620
покупка своего жилья                      620
ремонт жилью                              612
покупка жилой недвижимости                607
на покупку своего автомобиля              505
заняться высшим образованием      

purpose — цель получения кредита

38 уникальных значений, правдоподобно, видна необходимость объединения в категории.

**Вывод**

Произведя обзор предоставленных данных можно сделать предварительный вывод о том, что количество и качество этих данных достаточны для проведения анализа.

Вместе с этим хотелось бы отметить несколько моментов:
1. Данные читаемы, наименования корректны, предоставленное описание соответствует содержанию.
2. В графах days_employed (общий трудовой стаж в днях) и total_income (ежемесячный доход) есть пропущенные значения. В обоих колонках заполнены по 19351 из 21525 значений, что составляет 89.9%.
3. Графа days_employed (общий трудовой стаж в днях) - данные в представленном виде использовать не представляется возможным, необходимо попытаться привести их к адекватному виду. В худшем случае, от этих данных придется отказаться, однако, цель исследования может быть достигнута и без них. В любом случае стоит рассмотреть в качестве отдельной задачи процесс формирования этих данных и выявить причину искажения информации.
4. В остальных графах проблем с данными не выявлено, либо они не критичны.

Приступим к устранению выявленных недостатков.

## Шаг 2. Предобработка данных

### Обработка пропусков

**Выявление закономерностей в пропущенных значениях.**

In [13]:
#подсчет пропусков
data.isna().sum()

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64

Мы можем заметить, что количество пропусков в обеих колонках совпадает. Проверим, не относятся ли эти пропуски к одним и тем же строкам.

In [14]:
# выведем строки с пропусками в колонке days_employed
data[data['days_employed'].isna()]

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
12,0,NaN,65,среднее,1,гражданский брак,1,M,пенсионер,0,NaN,сыграть свадьбу
26,0,NaN,41,среднее,1,женат / замужем,0,M,госслужащий,0,NaN,образование
29,0,NaN,63,среднее,1,Не женат / не замужем,4,F,пенсионер,0,NaN,строительство жилой недвижимости
41,0,NaN,50,среднее,1,женат / замужем,0,F,госслужащий,0,NaN,сделка с подержанным автомобилем
55,0,NaN,54,среднее,1,гражданский брак,1,F,пенсионер,1,NaN,сыграть свадьбу
...,...,...,...,...,...,...,...,...,...,...,...,...
21489,2,NaN,47,Среднее,1,женат / замужем,0,M,компаньон,0,NaN,сделка с автомобилем
21495,1,NaN,50,среднее,1,гражданский брак,1,F,сотрудник,0,NaN,свадьба
21497,0,NaN,48,ВЫСШЕЕ,0,женат / замужем,0,F,компаньон,0,NaN,строительство недвижимости
21502,1,NaN,42,среднее,1,женат / замужем,0,F,сотрудник,0,NaN,строительство жилой недвижимости


In [15]:
# выведем строки с пропусками в колонке total_income
data[data['total_income'].isna()]

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
12,0,NaN,65,среднее,1,гражданский брак,1,M,пенсионер,0,NaN,сыграть свадьбу
26,0,NaN,41,среднее,1,женат / замужем,0,M,госслужащий,0,NaN,образование
29,0,NaN,63,среднее,1,Не женат / не замужем,4,F,пенсионер,0,NaN,строительство жилой недвижимости
41,0,NaN,50,среднее,1,женат / замужем,0,F,госслужащий,0,NaN,сделка с подержанным автомобилем
55,0,NaN,54,среднее,1,гражданский брак,1,F,пенсионер,1,NaN,сыграть свадьбу
...,...,...,...,...,...,...,...,...,...,...,...,...
21489,2,NaN,47,Среднее,1,женат / замужем,0,M,компаньон,0,NaN,сделка с автомобилем
21495,1,NaN,50,среднее,1,гражданский брак,1,F,сотрудник,0,NaN,свадьба
21497,0,NaN,48,ВЫСШЕЕ,0,женат / замужем,0,F,компаньон,0,NaN,строительство недвижимости
21502,1,NaN,42,среднее,1,женат / замужем,0,F,сотрудник,0,NaN,строительство жилой недвижимости


Наше предположение подтвердилось: данные по доходу отсутствуют в тех же строках, в которых отсутствуют и данные о стаже работы. Эта информация должна помочь обнаружить причину отсутствия данных.

Таких записей довольно много - 2174, терять такой объем данных нежелательно, поэтому заполним пропуски.
Поскольку обе графы являются количественными, заполнять будем средним или медианой, предварительно их сравнив.

**Приступим к заполнению пропусков в графе 'total_income'.**

In [16]:
# сгруппируем таблицу по типу занятости и выведем агрегированные данные столбцу total_income
data.groupby('income_type')['total_income'].agg(['median', 'mean', 'count'])

,median,mean,count
income_type,,,
безработный,131339.751676,131339.751676,2
в декрете,53829.130729,53829.130729,1
госслужащий,150447.935283,170898.309923,1312
компаньон,172357.950966,202417.461462,4577
пенсионер,118514.486412,137127.465690,3443
предприниматель,499163.144947,499163.144947,1
сотрудник,142594.396847,161380.260488,10014
студент,98201.625314,98201.625314,1


Сравниваем медиану и среднее. Заменять будем на медиану, поскольку она более точно описывает пропущенные значения.

Произведем замену пропусков дохода на медианное значение этой величины для каждого типа занятости.

In [17]:
# перезаписываем столбец total_income с заменой пропусков на медианное значение в группе
data["total_income"] = data.groupby("income_type")["total_income"].transform(lambda x: x.fillna(x.median()))

In [18]:
# проверяем, остались ли пропуски в графе total_income
data.isna().sum()

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income           0
purpose                0
dtype: int64

In [19]:
# проверяем, не изменились ли медианные значения в графе total_income
data.groupby('income_type')['total_income'].median()

income_type
безработный        131339.751676
в декрете           53829.130729
госслужащий        150447.935283
компаньон          172357.950966
пенсионер          118514.486412
предприниматель    499163.144947
сотрудник          142594.396847
студент             98201.625314
Name: total_income, dtype: float64

Проверили результат замены: количество пустых - 0, медианы не изменились. Вывод: замены проведены на всем объеме данных, значения подставлены верные.

**Приступим к заполнению пропусков в графе 'days_employed'.**

Изучим значения в графе days_employed более пристально, поскольку пути замены пропусков не очевидны, а значения величин вызывают сомнения.

In [20]:
# выведем сведения о распределении значений в графе days_employed
data['days_employed'].describe()

count     19351.000000
mean      63046.497661
std      140827.311974
min      -18388.949901
25%       -2747.423625
50%       -1203.369529
75%        -291.095954
max      401755.400475
Name: days_employed, dtype: float64

Видим, что имеются как положительные, так отрицательные значения, что противоречит здравому смыслу. Попробуем разделить выборку по этому признаку и проанализировать каждую из получившихся выборок по отдельности.

**Рассмотрим положительные величины в графе days_employed**

In [21]:
# выведем сведения о распределении значений в датасете при положительных величинах в графе days_employed
data[data['days_employed'] > 0].describe(include='all')

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
count,3445.000000,3445.000000,3445.000000,3445,3445.000000,3445,3445.000000,3445,3445,3445.000000,3445.000000,3445
unique,NaN,NaN,NaN,14,NaN,5,NaN,2,2,NaN,NaN,38
top,NaN,NaN,NaN,среднее,NaN,женат / замужем,NaN,F,пенсионер,NaN,NaN,сыграть свадьбу
freq,NaN,NaN,NaN,2511,NaN,1873,NaN,2807,3443,NaN,NaN,127
mean,0.128302,365004.309916,59.124819,NaN,0.914659,NaN,0.984325,NaN,NaN,0.052830,137124.105624,NaN
std,0.955042,21075.016396,7.580584,NaN,0.517103,NaN,1.316071,NaN,NaN,0.223727,80242.210917,NaN
min,-1.000000,328728.720605,0.000000,NaN,0.000000,NaN,0.000000,NaN,NaN,0.000000,20667.263793,NaN
25%,0.000000,346639.413916,56.000000,NaN,1.000000,NaN,0.000000,NaN,NaN,0.000000,82876.335652,NaN
50%,0.000000,365213.306266,60.000000,NaN,1.000000,NaN,0.000000,NaN,NaN,0.000000,118514.486412,NaN
75%,0.000000,383246.444219,64.000000,NaN,1.000000,NaN,2.000000,NaN,NaN,0.000000,169746.263276,NaN


Видим, что значения распределены сходно с полным датасетом за исключение графы income_type: тут осталось только 2 значения из 8. Выясним, что это за значения и их распределение. 

In [22]:
# выведем уникальные значения в графе income_type
print(data[data['days_employed'] > 0]['income_type'].unique())

['пенсионер' 'безработный']


In [23]:
# выведем количество уникальных значений в графе income_type
data[data['days_employed'] > 0]['income_type'].value_counts()

пенсионер      3443
безработный       2
Name: income_type, dtype: int64

Можем заметить, что положительные значения трудового стажа приходятся на пенсионеров (и 2х безработных), т.е. на тех, кто в данный момент не работает.

Теперь обратим внимание на величину значений.

In [24]:
# выведем сведения о распределении положительных значений в графе days_employed
data[data['days_employed'] > 0]['days_employed'].describe()

count      3445.000000
mean     365004.309916
std       21075.016396
min      328728.720605
25%      346639.413916
50%      365213.306266
75%      383246.444219
max      401755.400475
Name: days_employed, dtype: float64

Чтобы было привычнее воспринимать данные о трудовом стаже переведем значения из дней (как заявлено в описании данных) в годы.

In [25]:
# переводим значения в графе days_employed из дней в годы
data[data['days_employed'] > 0]['days_employed'].describe() / 365

count       9.438356
mean     1000.011808
std        57.739771
min       900.626632
25%       949.697024
50%      1000.584401
75%      1049.990258
max      1100.699727
Name: days_employed, dtype: float64

Значения распределены в диапазоне от 900 до 1100 лет. Столько не живут.

Возможно, при выгрузке данных произошло искажение единиц измерения и данные представлены в часах. Проверим предположение.

In [26]:
# переводим значения в графе days_employed из часов в годы
data[data['days_employed'] > 0]['days_employed'].describe() / (365 * 24)

count     0.393265
mean     41.667159
std       2.405824
min      37.526110
25%      39.570709
50%      41.691017
75%      43.749594
max      45.862489
Name: days_employed, dtype: float64

Значения распределены в диапазоне от 37 до 45 лет, что является типичным для пенсионеров.

Величина значения приобретает смысл, если считать, что стаж учитывается в часах, а не днях, как указано в описании.

In [27]:
# выведем агрегированные величины трудового стажа, предварительно сгруппировав данные по возрасту
data[data['days_employed'] > 0].groupby('dob_years')['days_employed'].agg(['median', 'mean', 'count'])

,median,mean,count
dob_years,,,
0,366067.781030,362537.515114,17
22,334764.259831,334764.259831,1
26,376872.682465,376872.682465,2
27,376824.585817,362032.797773,3
28,350340.760224,350340.760224,1
31,337524.466835,337524.466835,1
32,336213.383598,339365.593129,3
33,365649.502024,365649.502024,2
34,391961.274017,386022.017215,3


In [28]:
# переводим days_employed в дни   
data.loc[data['days_employed'] > 0, 'days_employed'] = data[data['days_employed'] > 0]['days_employed'].transform(lambda x: x / 24)

In [29]:
# проверяем успешность преобразования
data[data['days_employed'] > 0].groupby('dob_years')['days_employed'].median()

dob_years
0     15252.824210
22    13948.510826
26    15703.028436
27    15701.024409
28    14597.531676
31    14063.519451
32    14008.890983
33    15235.395918
34    16331.719751
35    14980.733146
36    15387.598283
37    14815.435224
38    14920.720116
39    15546.460469
40    15441.497734
41    15233.563520
42    14584.425847
43    14521.896675
44    15161.641656
45    15550.838500
46    15668.158672
47    15311.558417
48    15781.323956
49    15096.705197
50    14965.817769
51    14858.674594
52    15053.774583
53    14909.020719
54    15209.049986
55    15288.583088
56    15277.430275
57    15200.251787
58    15229.798854
59    15260.454727
60    15175.463929
61    15094.423719
62    15236.016258
63    15250.839886
64    15275.708222
65    15260.055052
66    15469.106688
67    15031.423450
68    15414.701987
69    15174.475213
70    15246.151361
71    15177.282066
72    15457.160702
73    15263.877042
74    14947.602918
Name: days_employed, dtype: float64

Положительные значения в графе days_employed уменьшились в 24 раза. Изменение данных прошло успешно.

Можем приступить к замещению пропусков в графе days_employed для неработающих заемщиков.

In [30]:
# проверяем количество пропусков для категорий 'пенсионер' и 'безработный'
data[(data['income_type'] == 'пенсионер') | (data['income_type'] == 'безработный')].isna().sum()

children              0
days_employed       413
dob_years             0
education             0
education_id          0
family_status         0
family_status_id      0
gender                0
income_type           0
debt                  0
total_income          0
purpose               0
dtype: int64

In [31]:
# заполняем пропуски медианными значениями с предварительной группировкой по возрасту и типу занятости
data.loc[data['income_type'] == 'пенсионер', 'days_employed'] = data[data['income_type'] == 'пенсионер'].groupby('dob_years')['days_employed'].transform(lambda x: x.fillna(x.median()))

In [32]:
# проверяем успешность преобразования
data[(data['income_type'] == 'пенсионер') | (data['income_type'] == 'безработный')].isna().sum()

children            0
days_employed       2
dob_years           0
education           0
education_id        0
family_status       0
family_status_id    0
gender              0
income_type         0
debt                0
total_income        0
purpose             0
dtype: int64

Осталось 2 незаполненных значения. Выведем строки с ними.

In [33]:
data[(data['days_employed'].isna()) & (data['income_type'] == 'пенсионер')]

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
3619,0,NaN,24,среднее,1,женат / замужем,0,F,пенсионер,0,118514.486412,покупка своего жилья
18175,2,NaN,31,среднее,1,гражданский брак,1,F,пенсионер,0,118514.486412,свадьба


Это строки, для которых отсутсвуют медианные значения по возрасту. Заполним их медианым значением для категории 'пенсионер'.

In [34]:
# заполняем пропуски без группировки по 'dob_years'
data.loc[data['income_type'] == 'пенсионер', 'days_employed'] = data[data['income_type'] == 'пенсионер']['days_employed'].transform(lambda x: x.fillna(x.median()))

In [35]:
# вывод незаполненных ячеек для категорий 'пенсионер' и 'безработный'
data[(data['income_type'] == 'пенсионер') | (data['income_type'] == 'безработный')].isna().sum()

children            0
days_employed       0
dob_years           0
education           0
education_id        0
family_status       0
family_status_id    0
gender              0
income_type         0
debt                0
total_income        0
purpose             0
dtype: int64

Незаполненных ячеек days_employed для категорий 'пенсионер' и 'безработный' не осталось.

In [36]:
# вывод незаполненных ячеек для всей таблицы
data.isna().sum()

children               0
days_employed       1761
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income           0
purpose                0
dtype: int64

Незаполненные ячеки days_employed для работающих категорий остались незаполненными.

In [37]:
# вывод медианных значений days_employed
data[data['days_employed'] > 0].groupby('dob_years')['days_employed'].median()

dob_years
0     15252.824210
22    13948.510826
24    15229.798854
26    15703.028436
27    15701.024409
28    14597.531676
31    14646.659153
32    14008.890983
33    15235.395918
34    16331.719751
35    14980.733146
36    15387.598283
37    14815.435224
38    14920.720116
39    15546.460469
40    15441.497734
41    15233.563520
42    14584.425847
43    14521.896675
44    15161.641656
45    15471.839685
46    15668.158672
47    15311.558417
48    15781.323956
49    15096.705197
50    14965.817769
51    14858.674594
52    15053.774583
53    14909.020719
54    15209.049986
55    15288.583088
56    15277.430275
57    15200.251787
58    15229.798854
59    15260.454727
60    15175.463929
61    15094.423719
62    15236.016258
63    15250.839886
64    15275.708222
65    15260.055052
66    15469.106688
67    15031.423450
68    15414.701987
69    15174.475213
70    15246.151361
71    15177.282066
72    15457.160702
73    15263.877042
74    14947.602918
Name: days_employed, dtype: float64

Медианные значения days_employed не изменились.

Проверка проведена: пропуски в графе days_employed для категорий 'пенсионер' и 'безработный' заполнены корректно.

**Рассмотрим отрицательные величины в графе days_employed**

In [38]:
# выведем сведения о распределении значений в датасете при отрицательных величинах в графе days_employed
data[data['days_employed'] < 0].describe(include='all')

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
count,15906.000000,15906.000000,15906.000000,15906,15906.000000,15906,15906.000000,15906,15906,15906.000000,1.590600e+04,15906
unique,NaN,NaN,NaN,14,NaN,5,NaN,3,6,NaN,NaN,38
top,NaN,NaN,NaN,среднее,NaN,женат / замужем,NaN,F,сотрудник,NaN,NaN,свадьба
freq,NaN,NaN,NaN,9831,NaN,9270,NaN,9945,10014,NaN,NaN,600
mean,0.625990,-2353.015932,39.818245,NaN,0.798378,NaN,0.969634,NaN,NaN,0.087326,1.739844e+05,NaN
std,1.430569,2304.243851,10.663171,NaN,0.554845,NaN,1.442263,NaN,NaN,0.282320,1.061303e+05,NaN
min,-1.000000,-18388.949901,0.000000,NaN,0.000000,NaN,0.000000,NaN,NaN,0.000000,2.136765e+04,NaN
25%,0.000000,-3157.480084,32.000000,NaN,0.000000,NaN,0.000000,NaN,NaN,0.000000,1.082713e+05,NaN
50%,0.000000,-1630.019381,39.000000,NaN,1.000000,NaN,0.000000,NaN,NaN,0.000000,1.511346e+05,NaN
75%,1.000000,-756.371964,48.000000,NaN,1.000000,NaN,1.000000,NaN,NaN,0.000000,2.114944e+05,NaN


In [39]:
# выведем уникальные значения в графе income_type
data[data['days_employed'] < 0]['income_type'].unique()

array(['сотрудник', 'компаньон', 'госслужащий', 'студент',
       'предприниматель', 'в декрете'], dtype=object)

Ещё раз получили подтверждение, что трудовой стаж со знаком минус указан для работающих в данный момент категорий заемщиков.

Теперь рассмотрим значения величин.

In [40]:
# выведем сведения о распределении отрицательных значений в графе days_employed
data[data['days_employed'] < 0]['days_employed'].describe()

count    15906.000000
mean     -2353.015932
std       2304.243851
min     -18388.949901
25%      -3157.480084
50%      -1630.019381
75%       -756.371964
max        -24.141633
Name: days_employed, dtype: float64

In [41]:
# преобразуем сведения о распределении в графе days_employed в годы
data[data['days_employed'] < 0]['days_employed'].describe() / 365

count    43.578082
mean     -6.446619
std       6.312997
min     -50.380685
25%      -8.650630
50%      -4.465807
75%      -2.072252
max      -0.066141
Name: days_employed, dtype: float64

Большинство значений лежит в диапазоне от 2 до 8 лет. Вероятно, речь идет о стаже на последнем месте работы.

In [42]:
# выведем агрегированные величины трудового стажа, предварительно сгруппировав данные по возрасту
data[data['days_employed'] < 0].groupby('dob_years')['days_employed'].agg(['median', 'mean', 'count'])

,median,mean,count
dob_years,,,
0,-1560.900431,-2200.375775,74
19,-724.492610,-633.678086,13
20,-674.838979,-684.944308,46
21,-618.733817,-709.440930,93
22,-699.061214,-781.376775,165
23,-690.204208,-827.309437,218
24,-947.731043,-1026.405485,243
25,-919.199388,-1088.406453,334
26,-1072.973567,-1200.288052,371


Посмотрев на распределение средних и медианных значений по возрастам, можно получить ещё одно подтверждение нашего предположения, что для работающих на данный момент клиентов банка учтен стаж на последнем месте работы.

In [43]:
# зафиксируем количество пропусков
data.isna().sum()

children               0
days_employed       1761
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income           0
purpose                0
dtype: int64

In [44]:
# заполняем медианами пропуски в графе days_employed для работающих категорий (за исключением 'пенсионер' и 'безработный')
# c предварительной группировкой по возрасту
data.loc[((data['income_type'] != 'пенсионер') & (data['income_type'] != 'безработный')), 'days_employed'] = (
    data[((data['income_type'] != 'пенсионер') & (data['income_type'] != 'безработный'))]
    .groupby('dob_years')['days_employed'].transform(lambda x: x.fillna(x.median()))
)

Проверим результат преобразования.

In [45]:
# выведем пропуски
data.isna().sum()

children            0
days_employed       0
dob_years           0
education           0
education_id        0
family_status       0
family_status_id    0
gender              0
income_type         0
debt                0
total_income        0
purpose             0
dtype: int64

In [46]:
# выведем агрегированные величины(в т.ч. медиану) графы days_employed с предварительной группировкой по возрасту
data[data['days_employed'] < 0].groupby('dob_years')['days_employed'].agg(['median', 'mean', 'count'])

,median,mean,count
dob_years,,,
0,-1560.900431,-2145.112473,81
19,-724.492610,-640.164838,14
20,-674.838979,-683.953590,51
21,-618.733817,-694.731668,111
22,-699.061214,-773.687959,182
23,-690.204208,-807.877199,254
24,-947.731043,-1020.422638,263
25,-919.199388,-1077.505157,357
26,-1072.973567,-1189.312666,406


Пропусков нет, медианы не изменились. Замена пропусков проведена корректно.

Приступим к коррекции значений.

In [47]:
# убираем минусы
data['days_employed'] = data['days_employed'].transform(lambda x: x.abs())

Проверим результат преобразований.

In [48]:
# выведем распределение значений
data['days_employed'].describe()

count    21525.000000
mean      4610.686199
std       5349.685498
min         24.141633
25%       1007.381548
50%       2113.346888
75%       5349.026537
max      18388.949901
Name: days_employed, dtype: float64

Значения положительные, величины адекватные. Преобразования проведены успешно.

In [49]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     21525 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      21525 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


Пропусков нет.

**Вывод**

Пропуски в таблице заполнены.

Отметим следующие нюансы:

1. Заполнение пропусков в графе income_type не вызывает опасений и этими данными можно пользоваться в дальнейшем.
2. Для заполнения пропусков в графе days_employed была использована масса допущений и предположений. Велика вероятность искажения информации. Более того, если все сделанные предположения верны, то величины в этой графе описывают два разных параметра: общий трудовой стаж для одних категорий заемщиков и трудовой стаж на последнем месте работы для других категорий. Исходя из изложенного, использование данных из этой графы нежелательно.
3. Информация для выявления причины искажения данных. Пропуски имеют место быть в двух столбцах (days_employed и income_type), при этом они затрагивают одни и те же записи. Имеются отрицательные значения days_employed - для работающих категорий заемщиков. Величины положительных значений days_employed (для неработающих категорий заемщиков) больше разумных в 24 раза (вероятно, указаны часы вместо дней).

### Замена типа данных

Выведем информацию об используемых типах данных.

In [50]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     21525 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      21525 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


В двух столбцах используется тип данных float64. Дробная часть не несет значимой информации, поэтому от нее можно избавится.

Для преобразования типа данных будем использовать метод astype(). Оставим дефолтное значение параметра errors='raise', что позволит нам сразу выявить проблемы при их наличии.

In [51]:
data['days_employed'] = data['days_employed'].astype('int')

In [52]:
data['total_income'] = data['total_income'].astype('int')

Также можем оптимизироать тип данных в графах debt, dob_years, education_id, family_status_id и children изменив подтип с int64 на int8.

In [53]:
data['dob_years'] = data['dob_years'].astype('int8')

In [54]:
data['education_id'] = data['education_id'].astype('int8')

In [55]:
data['family_status_id'] = data['family_status_id'].astype('int8')

In [56]:
data['children'] = data['children'].astype('int8')

В графе debt изменим тип данных на bool.

In [57]:
data['debt'] = data['debt'].astype('int8')

Проверим результат преобразований.

In [60]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   children          21525 non-null  int8  
 1   days_employed     21525 non-null  int32 
 2   dob_years         21525 non-null  int8  
 3   education         21525 non-null  object
 4   education_id      21525 non-null  int8  
 5   family_status     21525 non-null  object
 6   family_status_id  21525 non-null  int8  
 7   gender            21525 non-null  object
 8   income_type       21525 non-null  object
 9   debt              21525 non-null  int8  
 10  total_income      21525 non-null  int32 
 11  purpose           21525 non-null  object
dtypes: int32(2), int8(5), object(5)
memory usage: 1.1+ MB


Преобразования проведены успешно.

**Вывод**

Произведена замена типа данных. В таблице остались столбцы типов int - 7 шт. и object - 5 шт. В результате объем используемой памяти сократился практически вдвое.

### Обработка дубликатов

Проверим наличие дубликатов в данных.

In [61]:
data.duplicated().sum()

54

В таблице нет каких-либо уникальных параметров, таких как id клиента или ФИО, поэтому небольшое количество совпадающих записей возможно. Удалять их некорректно.

Рассмотрим значения в каждой графе и избавимся от лишних.

Графа children

In [62]:
data['children'].value_counts()

 0     14149
 1      4818
 2      2055
 3       330
 20       76
-1        47
 4        41
 5         9
Name: children, dtype: int64

Значение -1 ребенок встречается 47 раз. Вероятно, имеет место опечатка при вводе данных. Заменим -1 на 1.

Значение 20 детей встречается 76 раз. Учитывая, что количество людей с определенным числом детей, уверенно снижается с увеличением числа детей и доходит максимум до 5 детей, делаем вывод, что 20 детей - также опечатка либо ошибка перевода из формата 2.0. Заменим 20 на 2.

In [63]:
data['children'] = data['children'].replace(-1, 1)

In [64]:
data['children'] = data['children'].replace(20, 2)

In [65]:
data['children'].value_counts()

0    14149
1     4865
2     2131
3      330
4       41
5        9
Name: children, dtype: int64

После преобразований значения в графе children корректные.

Рассмотрим графу dob_years.

In [66]:
data['dob_years'].value_counts()

35    617
40    609
41    607
34    603
38    598
42    597
33    581
39    573
31    560
36    555
44    547
29    545
30    540
48    538
37    537
50    514
43    513
32    510
49    508
28    503
45    497
27    493
56    487
52    484
47    480
54    479
46    475
58    461
57    460
53    459
51    448
59    444
55    443
26    408
60    377
25    357
61    355
62    352
63    269
64    265
24    264
23    254
65    194
22    183
66    183
67    167
21    111
0     101
68     99
69     85
70     65
71     58
20     51
72     33
19     14
73      8
74      6
75      1
Name: dob_years, dtype: int64

Имеется одно некорректное значение - 0. Рассмотрим его подробнее.

In [67]:
data[data['dob_years'] == 0].describe()

,children,days_employed,dob_years,education_id,family_status_id,debt,total_income
count,101.000000,101.000000,101.0,101.000000,101.000000,101.000000,101.000000
mean,0.495050,4715.405941,0.0,0.673267,1.237624,0.079208,156652.801980
std,0.807759,5541.582352,0.0,0.512033,1.524129,0.271410,70995.969572
min,0.000000,108.000000,0.0,0.000000,0.000000,0.000000,34974.000000
25%,0.000000,1018.000000,0.0,0.000000,0.000000,0.000000,102621.000000
50%,0.000000,1720.000000,0.0,1.000000,1.000000,0.000000,142594.000000
75%,1.000000,5043.000000,0.0,1.000000,3.000000,0.000000,201852.000000
max,3.000000,16708.000000,0.0,2.000000,4.000000,1.000000,386373.000000


In [68]:
data[data['dob_years'] == 0].describe(include='object')

,education,family_status,gender,income_type,purpose
count,101,101,101,101,101
unique,7,5,2,4,35
top,среднее,женат / замужем,F,сотрудник,жилье
freq,59,49,72,55,6


In [69]:
data[data['dob_years'] == 0]['income_type'].unique()

array(['пенсионер', 'сотрудник', 'компаньон', 'госслужащий'], dtype=object)

In [70]:
data[data['dob_years'] == 0]['income_type'].value_counts()

сотрудник      55
пенсионер      20
компаньон      20
госслужащий     6
Name: income_type, dtype: int64

Каких-либо закономерностей в записях со значением dob_years=0 не выявлено, заменим его медианой.

In [71]:
dob_years_median = data['dob_years'].median()
dob_years_median

42.0

In [72]:
data['dob_years'] = data['dob_years'].replace(0, dob_years_median)

In [73]:
data['dob_years'].describe()

count    21525.000000
mean        43.490453
std         12.218595
min         19.000000
25%         34.000000
50%         42.000000
75%         53.000000
max         75.000000
Name: dob_years, dtype: float64

После преобразований значения в графе dob_years корректные.

Рассмотрим графу education.

In [74]:
data.groupby('education_id')['education'].value_counts()

education_id  education          
0             высшее                  4718
              ВЫСШЕЕ                   274
              Высшее                   268
1             среднее                13750
              СРЕДНЕЕ                  772
              Среднее                  711
2             неоконченное высшее      668
              Неоконченное высшее       47
              НЕОКОНЧЕННОЕ ВЫСШЕЕ       29
3             начальное                250
              НАЧАЛЬНОЕ                 17
              Начальное                 15
4             ученая степень             4
              УЧЕНАЯ СТЕПЕНЬ             1
              Ученая степень             1
Name: education, dtype: int64

Видим большое количество дублей. Причина их возникновения - использование различных регистров при вводе данных. Избавимся от дублей путем приведения всех значений к нижнему регистру.

In [75]:
data['education'] = data['education'].str.lower()

In [76]:
data.groupby('education_id')['education'].value_counts()

education_id  education          
0             высшее                  5260
1             среднее                15233
2             неоконченное высшее      744
3             начальное                282
4             ученая степень             6
Name: education, dtype: int64

После преобразований значения в графе education корректные.

Рассмотрим графу family_status.

In [77]:
data.groupby('family_status_id')['family_status'].value_counts()

family_status_id  family_status        
0                 женат / замужем          12380
1                 гражданский брак          4177
2                 вдовец / вдова             960
3                 в разводе                 1195
4                 Не женат / не замужем     2813
Name: family_status, dtype: int64

Дублей нет. Однако, для единообразия приведем и здесь все значения к нижнему регистру.

In [78]:
data['family_status'] = data['family_status'].str.lower()

In [79]:
data.groupby('family_status_id')['family_status'].value_counts()

family_status_id  family_status        
0                 женат / замужем          12380
1                 гражданский брак          4177
2                 вдовец / вдова             960
3                 в разводе                 1195
4                 не женат / не замужем     2813
Name: family_status, dtype: int64

После преобразований значения в графе education корректные.

Рассмотрим графу gender.

In [80]:
data['gender'].value_counts()

F      14236
M       7288
XNA        1
Name: gender, dtype: int64

Имеется одно некорректное значение, рассмотрим его подробнее.

In [81]:
data[data['gender'] == 'XNA']

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
10701,0,2358,24,неоконченное высшее,2,гражданский брак,1,XNA,компаньон,0,203905,покупка недвижимости


Каких-либо аномалий не наблюдается, заменим некорректное значение на наиболее частотное.

In [82]:
data['gender'] = data['gender'].replace('XNA', 'F')

In [83]:
data['gender'].value_counts()

F    14237
M     7288
Name: gender, dtype: int64

После преобразований значения в графе education корректные.

Рассмотрим графу income_type.

In [84]:
data['income_type'].value_counts()

сотрудник          11119
компаньон           5085
пенсионер           3856
госслужащий         1459
безработный            2
предприниматель        2
студент                1
в декрете              1
Name: income_type, dtype: int64

Возможно, "компаньон" и "предприниматель" это два разных названия одного вида деятельности. Проверим по доходам.

In [85]:
data.groupby('income_type')['total_income'].mean().sort_values()

income_type
в декрете           53829.000000
студент             98201.000000
безработный        131339.000000
пенсионер          135133.413641
сотрудник          159512.838205
госслужащий        168837.299520
компаньон          199413.927237
предприниматель    499163.000000
Name: total_income, dtype: float64

Доходы этих категорий отличаются в 2,5 раза. Объединять не стоит.

Значения в графе income_type корректные.

Рассмотрим графу purpose.

In [86]:
data['purpose'].value_counts()

свадьба                                   797
на проведение свадьбы                     777
сыграть свадьбу                           774
операции с недвижимостью                  676
покупка коммерческой недвижимости         664
покупка жилья для сдачи                   653
операции с жильем                         653
операции с коммерческой недвижимостью     651
покупка жилья                             647
жилье                                     647
покупка жилья для семьи                   641
строительство собственной недвижимости    635
недвижимость                              634
операции со своей недвижимостью           630
строительство жилой недвижимости          626
покупка недвижимости                      624
строительство недвижимости                620
покупка своего жилья                      620
ремонт жилью                              612
покупка жилой недвижимости                607
на покупку своего автомобиля              505
заняться высшим образованием      

Замечаний к написанию нет, для дальнейшей обработки данной информации необходима лемматизация.

Проверим итоговое количество дубликатов после проведенных преобразований.

In [87]:
data.duplicated().sum()

71

Общее количество дубликатов увеличилось с 54 до 71. Такое увеличение соответствует ожиданиям и объясняется упорядочиванием данных в колонках education и dob_years. 

Ранее высказанное утверждение о том, что в таблице нет каких-либо уникальных параметров, таких как id клиента или ФИО, поэтому небольшое количество совпадающих записей возможно и удалять их некорректно, остается в силе.

**Вывод**

Обработка дубликатов и некорректных значений проведена, данные подготовлены для дальнейшей оптимизации.

### Лемматизация

Для обработки информации из графы purpose необходимо провести лемматизацию. Это позволит объединить в категории цели получения кредита.

Процесс лемматизации затратный с точки зрения использования ресурсов компьютера, поэтому составим список уникальных значений интересующей категории и дальше будем работать уже с ним.

In [88]:
purpose_list = data['purpose'].unique()
purpose_list

array(['покупка жилья', 'приобретение автомобиля',
       'дополнительное образование', 'сыграть свадьбу',
       'операции с жильем', 'образование', 'на проведение свадьбы',
       'покупка жилья для семьи', 'покупка недвижимости',
       'покупка коммерческой недвижимости', 'покупка жилой недвижимости',
       'строительство собственной недвижимости', 'недвижимость',
       'строительство недвижимости', 'на покупку подержанного автомобиля',
       'на покупку своего автомобиля',
       'операции с коммерческой недвижимостью',
       'строительство жилой недвижимости', 'жилье',
       'операции со своей недвижимостью', 'автомобили',
       'заняться образованием', 'сделка с подержанным автомобилем',
       'получение образования', 'автомобиль', 'свадьба',
       'получение дополнительного образования', 'покупка своего жилья',
       'операции с недвижимостью', 'получение высшего образования',
       'свой автомобиль', 'сделка с автомобилем',
       'профильное образование', 'высшее об

Отберем слова, которые чаще всего используются для указания цели получения кредита.

In [89]:
# создаем пустой список
lemmas_list = []

In [90]:
# добавляем в созданный список леммы всех слов из списка целей
for purpose in purpose_list:
    for word in purpose.split():
        lemma = m.lemmatize(word)
        lemmas_list.append(lemma[0])           

In [91]:
# выведем получившийся список лемм
lemmas_list

['покупка',
 'жилье',
 'приобретение',
 'автомобиль',
 'дополнительный',
 'образование',
 'сыграть',
 'свадьба',
 'операция',
 'с',
 'жилье',
 'образование',
 'на',
 'проведение',
 'свадьба',
 'покупка',
 'жилье',
 'для',
 'семья',
 'покупка',
 'недвижимость',
 'покупка',
 'коммерческий',
 'недвижимость',
 'покупка',
 'жилой',
 'недвижимость',
 'строительство',
 'собственный',
 'недвижимость',
 'недвижимость',
 'строительство',
 'недвижимость',
 'на',
 'покупка',
 'подержать',
 'автомобиль',
 'на',
 'покупка',
 'свой',
 'автомобиль',
 'операция',
 'с',
 'коммерческий',
 'недвижимость',
 'строительство',
 'жилой',
 'недвижимость',
 'жилье',
 'операция',
 'со',
 'свой',
 'недвижимость',
 'автомобиль',
 'заниматься',
 'образование',
 'сделка',
 'с',
 'подержать',
 'автомобиль',
 'получение',
 'образование',
 'автомобиль',
 'свадьба',
 'получение',
 'дополнительный',
 'образование',
 'покупка',
 'свой',
 'жилье',
 'операция',
 'с',
 'недвижимость',
 'получение',
 'высокий',
 'образование',

In [92]:
# выведем список лемм с указанием частоты использования
Counter(lemmas_list)

Counter({'покупка': 10,
         'жилье': 7,
         'приобретение': 1,
         'автомобиль': 9,
         'дополнительный': 2,
         'образование': 9,
         'сыграть': 1,
         'свадьба': 3,
         'операция': 4,
         'с': 5,
         'на': 4,
         'проведение': 1,
         'для': 2,
         'семья': 1,
         'недвижимость': 10,
         'коммерческий': 2,
         'жилой': 2,
         'строительство': 3,
         'собственный': 1,
         'подержать': 2,
         'свой': 4,
         'со': 1,
         'заниматься': 2,
         'сделка': 2,
         'получение': 3,
         'высокий': 3,
         'профильный': 1,
         'сдача': 1,
         'ремонт': 1})

Выберем из списка уникальных слов те, которые могут являться категориями цели получения кредита. После удаления лишнего остался список из 5 слов: 'жилье', 'автомобиль', 'образование', 'свадьба', 'недвижимость'. Категории 'жилье' и 'недвижимость' можно объединить в одну. Итого получили 4 категории: 'автомобиль', 'образование', 'свадьба' и 'недвижимость'. На случай, если не одна из категорий не будет описывать цель кредита дополним список категорией 'другое'.

Напишем функцию, для определения категории кредита: 'автомобиль', 'образование', 'свадьба', 'недвижимость', 'другое'.

In [93]:
def purpose_to_category(purpose):
    """
    Возвращает категорию кредита по наименованию цели purpose.
    """
    words = m.lemmatize(purpose)
    if 'недвижимость' in words:
        return 'недвижимость'
    if 'жилье' in words:
        return 'недвижимость'
    if 'автомобиль' in words:
        return 'автомобиль'
    if 'образование' in words:
        return 'образование'
    if 'свадьба' in words:
        return 'свадьба'
    return 'другое' 

In [94]:
# проверим работу функции
print(purpose_to_category('строительство жилой недвижимости'))

недвижимость


Используя полученные данные создадим таблицу соответствия цели кредита и его категории.

In [95]:
# создаем таблицу с перечнем целей
purpose_category_dict = pd.DataFrame(data = purpose_list, columns = ['purpose'])

In [96]:
# используя функцию purpose_to_category добавляем в таблицу столбец с указанием категории кредита
purpose_category_dict['purpose_category'] = purpose_category_dict['purpose'].apply(purpose_to_category)

In [97]:
# выведем таблицу для проверки результата преобразований
purpose_category_dict

,purpose,purpose_category
0,покупка жилья,недвижимость
1,приобретение автомобиля,автомобиль
2,дополнительное образование,образование
3,сыграть свадьбу,свадьба
4,операции с жильем,недвижимость
5,образование,образование
6,на проведение свадьбы,свадьба
7,покупка жилья для семьи,недвижимость
8,покупка недвижимости,недвижимость
9,покупка коммерческой недвижимости,недвижимость


Категории определены корректно.

**Вывод**

С помощью лемматизации нам удалось выделить категории целей получения кредита и составить таблицу соответствия целей и категорий.

### Категоризация данных

Приступим к категориации данных. Мы уже сформировали категории по цели получения кредита, теперь необходимо дополнить имеющиеся  данные этим параметром. Создадим новую таблицу прибавив к основному датасету столбцы с указанием категорий.

In [98]:
# создаем таблицу с категориями, 
# первым шагом добавляем столбец purpose_category путем объединения таблиц data и purpose_category_dict
data_category = data.merge(purpose_category_dict, on='purpose', how='left')
data_category

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,purpose_category
0,1,8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,недвижимость
1,1,4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,автомобиль
2,0,5623,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья,недвижимость
3,3,4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование,образование
4,0,14177,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,свадьба
...,...,...,...,...,...,...,...,...,...,...,...,...,...
21520,1,4529,43,среднее,1,гражданский брак,1,F,компаньон,0,224791,операции с жильем,недвижимость
21521,0,14330,67,среднее,1,женат / замужем,0,F,пенсионер,0,155999,сделка с автомобилем,автомобиль
21522,1,2113,38,среднее,1,гражданский брак,1,M,сотрудник,1,89672,недвижимость,недвижимость
21523,3,3112,38,среднее,1,женат / замужем,0,M,сотрудник,1,244093,на покупку своего автомобиля,автомобиль


Проверяем успешность преобразований.

In [99]:
data_category['purpose_category'].value_counts()

недвижимость    10840
автомобиль       4315
образование      4022
свадьба          2348
Name: purpose_category, dtype: int64

Новая таблица создана, столбец purpose_category заполнен корректно.

Теперь приступим к разбивке на категории количественных параметров. 

Начнем с колонки total_income.

In [100]:
# выведем распределение значений total_income
data_category['total_income'].describe()

count    2.152500e+04
mean     1.652248e+05
std      9.804367e+04
min      2.066700e+04
25%      1.077980e+05
50%      1.425940e+05
75%      1.955490e+05
max      2.265604e+06
Name: total_income, dtype: float64

Напишем функцию для определения категории ежемесячного дохода.

Вначале разобьем диапазон значений на три категории: '100 тысяч и меньше', '101 - 200 тысяч' и 'больше 200 тысяч'.

In [101]:
def income_group(income):
    
    if income <= 100000:
        return '100 тысяч и меньше'
    if income <= 200000:
        return '101 - 200 тысяч'
    if income > 200000:
        return 'больше 200 тысяч'
    return 'неизвестно' 

In [102]:
# проверяем работу функции
income_group(300000)

'больше 200 тысяч'

С помощью написанной функции создадим в таблице столбец с категорией дохода.

In [103]:
data_category['income_group'] = data_category['total_income'].apply(income_group)
data_category

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,purpose_category,income_group
0,1,8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,недвижимость,больше 200 тысяч
1,1,4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,автомобиль,101 - 200 тысяч
2,0,5623,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья,недвижимость,101 - 200 тысяч
3,3,4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование,образование,больше 200 тысяч
4,0,14177,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,свадьба,101 - 200 тысяч
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21520,1,4529,43,среднее,1,гражданский брак,1,F,компаньон,0,224791,операции с жильем,недвижимость,больше 200 тысяч
21521,0,14330,67,среднее,1,женат / замужем,0,F,пенсионер,0,155999,сделка с автомобилем,автомобиль,101 - 200 тысяч
21522,1,2113,38,среднее,1,гражданский брак,1,M,сотрудник,1,89672,недвижимость,недвижимость,100 тысяч и меньше
21523,3,3112,38,среднее,1,женат / замужем,0,M,сотрудник,1,244093,на покупку своего автомобиля,автомобиль,больше 200 тысяч


Проверим результат преобразований.

In [104]:
# выведем распределение значений income_group
data_category['income_group'].value_counts()

101 - 200 тысяч       11995
больше 200 тысяч       5067
100 тысяч и меньше     4463
Name: income_group, dtype: int64

Получившееся распределение не информативно для нижнего и верхнего сегментов.

Чтобы выделить дополнительные категории, рассмотрим распределение значений в каждой группе.

In [105]:
# выведем распределение значений в категории '101 - 200 тысяч'
data_category[data_category['income_group'] == '101 - 200 тысяч']['total_income'].describe()

count     11995.000000
mean     144844.713631
std       26311.500949
min      100006.000000
25%      121931.000000
50%      142594.000000
75%      166395.000000
max      199980.000000
Name: total_income, dtype: float64

In [106]:
# выведем распределение значений в категории 'больше 200 тысяч'
data_category[data_category['income_group'] == 'больше 200 тысяч']['total_income'].describe()

count    5.067000e+03
mean     2.917557e+05
std      1.224777e+05
min      2.000010e+05
25%      2.245185e+05
50%      2.579940e+05
75%      3.165675e+05
max      2.265604e+06
Name: total_income, dtype: float64

In [107]:
# выведем распределение значений в категории '100 тысяч и меньше'
data_category[data_category['income_group'] == '100 тысяч и меньше']['total_income'].describe()

count     4463.000000
mean     76344.530585
std      16682.951725
min      20667.000000
25%      65189.000000
50%      79505.000000
75%      89983.000000
max      99998.000000
Name: total_income, dtype: float64

Перепишем функцию с добавлением дополнительных категорий.

In [108]:
def income_group(income):
    """
    Возвращает категорию ежемесячного дохода.
    """
    
    if income <= 50000:
        return '50 тысяч и меньше'
    if income <= 75000:
        return '51 - 75 тысяч'
    if income <= 100000:
        return '76 - 100 тысяч'
    if income <= 150000:
        return '101 - 150 тысяч'
    if income <= 200000:
        return '151 - 200 тысяч'
    if income <= 250000:
        return '201 - 250 тысяч'
    if income <= 300000:
        return '251 - 300 тысяч'
    if income <= 350000:
        return '301 - 350 тысяч'
    if income <= 400000:
        return '351 - 400 тысяч'
    if income <= 500000:
        return '401 - 500 тысяч'
    if income > 500000:
        return 'больше 500 тысяч'
    return 'неизвестно' 

С помощью написанной функции повторно создадим в таблице столбец с категорией дохода.

In [109]:
data_category['income_group'] = data_category['total_income'].apply(income_group)
data_category

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,purpose_category,income_group
0,1,8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,недвижимость,251 - 300 тысяч
1,1,4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,автомобиль,101 - 150 тысяч
2,0,5623,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья,недвижимость,101 - 150 тысяч
3,3,4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование,образование,251 - 300 тысяч
4,0,14177,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,свадьба,151 - 200 тысяч
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21520,1,4529,43,среднее,1,гражданский брак,1,F,компаньон,0,224791,операции с жильем,недвижимость,201 - 250 тысяч
21521,0,14330,67,среднее,1,женат / замужем,0,F,пенсионер,0,155999,сделка с автомобилем,автомобиль,151 - 200 тысяч
21522,1,2113,38,среднее,1,гражданский брак,1,M,сотрудник,1,89672,недвижимость,недвижимость,76 - 100 тысяч
21523,3,3112,38,среднее,1,женат / замужем,0,M,сотрудник,1,244093,на покупку своего автомобиля,автомобиль,201 - 250 тысяч


Проверим результат преобразований.

In [110]:
# выведем распределение значений income_group
data_category['income_group'].value_counts()

101 - 150 тысяч      7222
151 - 200 тысяч      4773
76 - 100 тысяч       2598
201 - 250 тысяч      2254
51 - 75 тысяч        1493
251 - 300 тысяч      1330
301 - 350 тысяч       624
50 тысяч и меньше     372
351 - 400 тысяч       330
401 - 500 тысяч       307
больше 500 тысяч      222
Name: income_group, dtype: int64

Получившееся распределение позволяет получить представление о всем диапазоне значений.

Выделим категории в графе dob_years.

In [111]:
# выведем значения категории и их количество
data_category['dob_years'].value_counts()

42    698
35    617
40    609
41    607
34    603
38    598
33    581
39    573
31    560
36    555
44    547
29    545
30    540
48    538
37    537
50    514
43    513
32    510
49    508
28    503
45    497
27    493
56    487
52    484
47    480
54    479
46    475
58    461
57    460
53    459
51    448
59    444
55    443
26    408
60    377
25    357
61    355
62    352
63    269
64    265
24    264
23    254
65    194
22    183
66    183
67    167
21    111
68     99
69     85
70     65
71     58
20     51
72     33
19     14
73      8
74      6
75      1
Name: dob_years, dtype: int64

Выделим категории по возрасту: "20 лет и младше", "старше 70 лет", между этими границами разобьем диапазон на группы по 5 лет.

Напишем функцию для определения возрастной категории.

In [112]:
def age_group(age):
    """
    Возвращает возврастную группу по значению возраста.
    """
    
    if age <= 20:
        return '20 лет и младше'
    if age <= 25:
        return '21 - 25 лет'
    if age <= 30:
        return '26 - 30 лет'
    if age <= 35:
        return '31 - 35 лет'
    if age <= 40:
        return '36 - 40 лет'
    if age <= 45:
        return '41 - 45 лет'
    if age <= 50:
        return '46 - 50 лет'
    if age <= 55:
        return '51 - 55 лет'
    if age <= 60:
        return '56 - 60 лет'
    if age <= 65:
        return '61 - 65 лет'
    if age <= 70:
        return '66 - 70 лет'
    if age > 70:
        return 'старше 70 лет'
    return 'неизвестен' 

In [113]:
# проверим работу функции
age_group(55)

'51 - 55 лет'

С помощью написанной функции создадим в таблице столбец с возрастной категорией.

In [114]:
data_category['age_group'] = data_category['dob_years'].apply(age_group)
data_category

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,purpose_category,income_group,age_group
0,1,8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,недвижимость,251 - 300 тысяч,41 - 45 лет
1,1,4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,автомобиль,101 - 150 тысяч,36 - 40 лет
2,0,5623,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья,недвижимость,101 - 150 тысяч,31 - 35 лет
3,3,4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование,образование,251 - 300 тысяч,31 - 35 лет
4,0,14177,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,свадьба,151 - 200 тысяч,51 - 55 лет
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21520,1,4529,43,среднее,1,гражданский брак,1,F,компаньон,0,224791,операции с жильем,недвижимость,201 - 250 тысяч,41 - 45 лет
21521,0,14330,67,среднее,1,женат / замужем,0,F,пенсионер,0,155999,сделка с автомобилем,автомобиль,151 - 200 тысяч,66 - 70 лет
21522,1,2113,38,среднее,1,гражданский брак,1,M,сотрудник,1,89672,недвижимость,недвижимость,76 - 100 тысяч,36 - 40 лет
21523,3,3112,38,среднее,1,женат / замужем,0,M,сотрудник,1,244093,на покупку своего автомобиля,автомобиль,201 - 250 тысяч,36 - 40 лет


Проверим результат разбивки по категориям.

In [115]:
data_category['age_group'].value_counts()

36 - 40 лет        2872
31 - 35 лет        2871
41 - 45 лет        2862
46 - 50 лет        2515
26 - 30 лет        2489
51 - 55 лет        2313
56 - 60 лет        2229
61 - 65 лет        1435
21 - 25 лет        1169
66 - 70 лет         599
старше 70 лет       106
20 лет и младше      65
Name: age_group, dtype: int64

Получившееся распределение позволяет получить представление о всем диапазоне значений.

Выделим категории в графе children.

In [116]:
# выведем значения категории и их количество
data_category['children'].value_counts()

0    14149
1     4865
2     2131
3      330
4       41
5        9
Name: children, dtype: int64

Видим, что количество заемщиков с четырьмя и пятью детьми 41 и 9 соответственно. Такое малое количество значений не дает возможности сделать статистически значимые выводы. Чтобы не потерять эти данные объединим заемщиков с 3, 4 и 5 детьми в одну категорию.

Напишем функцию для определения категории по количеству детей.

In [117]:
def children_group(children):
    """
    Возвращает группу по количеству детей.
    """
    
    if children == 0:
        return 'нет детей'
    if children == 1:
        return '1 ребенок'
    if children == 2:
        return '2 детей'
    if children >= 3:
        return '3 детей и более'
    return 'неизвестно' 

In [118]:
# проверим работу функции
children_group(2)

'2 детей'

С помощью написанной функции создадим в таблице столбец с категорией по количеству детей.

In [119]:
data_category['children_group'] = data_category['children'].apply(children_group)
data_category

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,purpose_category,income_group,age_group,children_group
0,1,8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,недвижимость,251 - 300 тысяч,41 - 45 лет,1 ребенок
1,1,4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,автомобиль,101 - 150 тысяч,36 - 40 лет,1 ребенок
2,0,5623,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья,недвижимость,101 - 150 тысяч,31 - 35 лет,нет детей
3,3,4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование,образование,251 - 300 тысяч,31 - 35 лет,3 детей и более
4,0,14177,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,свадьба,151 - 200 тысяч,51 - 55 лет,нет детей
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21520,1,4529,43,среднее,1,гражданский брак,1,F,компаньон,0,224791,операции с жильем,недвижимость,201 - 250 тысяч,41 - 45 лет,1 ребенок
21521,0,14330,67,среднее,1,женат / замужем,0,F,пенсионер,0,155999,сделка с автомобилем,автомобиль,151 - 200 тысяч,66 - 70 лет,нет детей
21522,1,2113,38,среднее,1,гражданский брак,1,M,сотрудник,1,89672,недвижимость,недвижимость,76 - 100 тысяч,36 - 40 лет,1 ребенок
21523,3,3112,38,среднее,1,женат / замужем,0,M,сотрудник,1,244093,на покупку своего автомобиля,автомобиль,201 - 250 тысяч,36 - 40 лет,3 детей и более


Проверим результат разбивки по категориям.

In [120]:
data_category['children_group'].value_counts()

нет детей          14149
1 ребенок           4865
2 детей             2131
3 детей и более      380
Name: children_group, dtype: int64

Получившееся распределение позволяет сделать статистически значимые выводы для каждой категории.

**Вывод**

Данные в столбцах children, dob_years, total_income и purpose отражают значимую информацию, достоверны, но использовать их в исходном виде не представляется возможным. Для того, чтобы извлечь пользу из этих данных была проведена категоризация. В таком виде данные пригодны для проведения анализа.	

## Шаг 3. Ответьте на вопросы

Вычислим среднюю вероятность просрочки по всей таблице.

In [121]:
debt_mean = data_category['debt'].mean()
debt_mean

0.08088269454123112

- **Есть ли зависимость между наличием детей и возвратом кредита в срок?**

Создадим сводную таблицу с помощью метода pivot_table()

Добавим новый параметр: relative_debt - отношение вероятности просрочки выбранной категории к средней вероятности просрочки по всей таблице.

In [122]:
# создаем таблицу показывающюю среднюю просрочку в завимости от значения children_group
children_pivot = data_category.pivot_table(index='children_group', values='debt', aggfunc='mean').reset_index()
# сортируем по возрастанию вероятности просрочки
children_pivot = children_pivot.sort_values(by = 'debt').reset_index(drop = True)
# добавляем параметр: relative_debt
children_pivot['relative_debt'] = children_pivot['debt'] / debt_mean
# переименовываем колонку debt на abs_debt
children_pivot = children_pivot.rename(columns={'debt': 'abs_debt'})
# приводим значения в удобочитаемую форму
children_pivot['abs_debt'] = children_pivot['abs_debt'].mul(100).round(2).astype(str).add(' %')
children_pivot['relative_debt'] = children_pivot['relative_debt'].round(2)
# выводим результат
children_pivot

,children_group,abs_debt,relative_debt
0,нет детей,7.51 %,0.93
1,3 детей и более,8.16 %,1.01
2,1 ребенок,9.15 %,1.13
3,2 детей,9.48 %,1.17


**Вывод**

Можем видеть, что зависимость между наличием детей и возвратом кредита в срок имеется.

Наибольший риск невозврата в срок у семей с двумя детьми. Риск уменьшается как с уменьшением, так и с увеличением количества детей и минимален при их отсутствии.

Вычисленные доли невозвратов в срок по каждой категории и отношение этой величины к среднему значению по всему объему данных представлены в таблице выше в графах abs_debt и relative_debt соответственно.

- **Есть ли зависимость между семейным положением и возвратом кредита в срок?**

Создадим сводную таблицу с помощью метода pivot_table()

Добавим новый параметр: relative_debt - отношение вероятности просрочки выбранной категории к средней вероятности просрочки по всей таблице.

In [127]:
# создаем таблицу показывающюю среднюю просрочку в завимости от значения family_status
family_status_pivot = data_category.pivot_table(index='family_status', values='debt', aggfunc='mean').reset_index()
# сортируем по возрастанию вероятности просрочки
family_status_pivot = family_status_pivot.sort_values(by = 'debt').reset_index(drop = True)
# добавляем параметр: relative_debt
family_status_pivot['relative_debt'] = family_status_pivot['debt'] / debt_mean
# переименовываем колонку debt на abs_debt
family_status_pivot = family_status_pivot.rename(columns={'debt': 'abs_debt'})
# приводим значения в удобочитаемую форму
family_status_pivot['abs_debt'] = family_status_pivot['abs_debt'].mul(100).round(2).astype(str).add(' %')
family_status_pivot['relative_debt'] = family_status_pivot['relative_debt'].round(2)
# выводим результат
family_status_pivot

,family_status,abs_debt,relative_debt
0,вдовец / вдова,6.56 %,0.81
1,в разводе,7.11 %,0.88
2,женат / замужем,7.52 %,0.93
3,гражданский брак,9.29 %,1.15
4,не женат / не замужем,9.74 %,1.20


**Вывод**

Можем видеть, что зависимость между семейным положением и возвратом кредита в срок имеется.

Наибольший риск невозврата в срок у категории "не женат / не замужем". Далее риск последовательно уменьшается у следующих категорий: "гражданский брак", "женат / замужем", "в разводе" и достигает минимального значения у  "вдовец / вдова".

Вычисленные доли невозвратов в срок по каждой категории и отношение этой величины к среднему значению по всему объему данных представлены в таблице выше в графах abs_debt и relative_debt соответственно.

- **Есть ли зависимость между уровнем дохода и возвратом кредита в срок?**

Создадим сводную таблицу с помощью метода pivot_table()

Добавим новый параметр: relative_debt - отношение вероятности просрочки выбранной категории к средней вероятности просрочки по всей таблице.

In [130]:
# создаем таблицу показывающюю среднюю просрочку в завимости от значения income_group
income_group_pivot = data_category.pivot_table(index='income_group', values='debt', aggfunc='mean').reset_index()
# сортируем по возрастанию вероятности просрочки
income_group_pivot = income_group_pivot.sort_values(by = 'debt').reset_index(drop = True)
# добавляем параметр: relative_debt
income_group_pivot['relative_debt'] = income_group_pivot['debt'] / debt_mean
# переименовываем колонку debt на abs_debt
income_group_pivot = income_group_pivot.rename(columns={'debt': 'abs_debt'})
# приводим значения в удобочитаемую форму
income_group_pivot['abs_debt'] = income_group_pivot['abs_debt'].mul(100).round(2).astype(str).add(' %')
income_group_pivot['relative_debt'] = income_group_pivot['relative_debt'].round(2)
# выводим результат
income_group_pivot

,income_group,abs_debt,relative_debt
0,401 - 500 тысяч,5.54 %,0.68
1,50 тысяч и меньше,6.18 %,0.76
2,больше 500 тысяч,6.31 %,0.78
3,251 - 300 тысяч,6.62 %,0.82
4,351 - 400 тысяч,7.27 %,0.90
5,201 - 250 тысяч,7.28 %,0.90
6,51 - 75 тысяч,7.57 %,0.94
7,301 - 350 тысяч,8.17 %,1.01
8,76 - 100 тысяч,8.39 %,1.04
9,151 - 200 тысяч,8.49 %,1.05


**Вывод**

Можем видеть, что зависимость между уровнем дохода и возвратом кредита в срок имеется.

Наибольший риск невозврата в срок у категорий со средним доходом и с доходом немного ниже среднего. Наименьший риск у категорий с высоким доходом, либо с экстремально низким. Общая закономерность такова, что риск максимален при среднем доходе и снижается как при уменьшении дохода, так и при его увеличении. Из этого выделяется категория дохода "301 - 350 тысяч", в котрой несмотря на высокий доход, риск невозврата в срок выше среднего.

Вычисленные доли невозвратов в срок по каждой категории и отношение этой величины к среднему значению по всему объему данных представлены в таблице выше в графах abs_debt и relative_debt соответственно.

- **Как разные цели кредита влияют на его возврат в срок?**

Создадим сводную таблицу с помощью метода pivot_table()

Добавим новый параметр: relative_debt - отношение вероятности просрочки выбранной категории к средней вероятности просрочки по всей таблице.

In [133]:
# создаем таблицу показывающюю среднюю просрочку в завимости от значения purpose_category
purpose_category_pivot = data_category.pivot_table(index='purpose_category', values='debt', aggfunc='mean').reset_index()
# сортируем по возрастанию вероятности просрочки
purpose_category_pivot = purpose_category_pivot.sort_values(by = 'debt').reset_index(drop = True)
# добавляем параметр: relative_debt
purpose_category_pivot['relative_debt'] = purpose_category_pivot['debt'] / debt_mean
# переименовываем колонку debt на abs_debt
purpose_category_pivot = purpose_category_pivot.rename(columns={'debt': 'abs_debt'})
# приводим значения в удобочитаемую форму
purpose_category_pivot['abs_debt'] = purpose_category_pivot['abs_debt'].mul(100).round(2).astype(str).add(' %')
purpose_category_pivot['relative_debt'] = purpose_category_pivot['relative_debt'].round(2)
# выводим результат
purpose_category_pivot

,purpose_category,abs_debt,relative_debt
0,недвижимость,7.21 %,0.89
1,свадьба,7.92 %,0.98
2,образование,9.2 %,1.14
3,автомобиль,9.34 %,1.15


**Вывод**

Можем видеть, что зависимость между целью кредита и возвратом кредита в срок имеется.

Наибольший риск невозврата в срок у категории "автомобиль", немного ниже у категории "образование", заметно ниже у категории "свадьба" и минимальный риск если целью кредита является недвижимость.

Вычисленные доли невозвратов в срок по каждой категории и отношение этой величины к среднему значению по всему объему данных представлены в таблице выше в графах abs_debt и relative_debt соответственно.

## Шаг 4. Общий вывод

Предоставленных данных было достаточно для достижения цели исследования - подготовки материалов для построения модели кредитного скоринга — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

Был проведен анализ набора данных о наличии просроченных платежей по 21525 заемщикам. Величина выборки позволяет говорить о значимости полученных выводов.

В результате анализа было установлено наличие зависимости возврата кредита в срок от следующих факторов:
- наличие детей
- семейное положение
- уровень дохода
- цель кредита

Были вычисленны вероятности возникновения просрочки по кредиту для каждого значения указаных выше параметров, а также отношение этой величины к средней просрочке по всему объему данных. Эти параметры могут быть использованы для построения модели кредитного скоринга.

Для возможности проведения анализа данные были предварительно обработаны: устранены пропуски, оптимизированы типы данных, дублирующиеся значения, проведена категоризация отдельных параметров.

Также были выявлены недостоверные категории данных и определены характеризующие их закономерности. Эта информация поможет определить причину возникновения искажений и внести коррективы в процесс сбора данных для исключения подобных ситуаций в будущем.

Учитывая изложенное, можем сделать вывод о том, что цель исследования достигнута, дальнейшие действия по развитию и оптимизации определены.

## Чек-лист готовности проекта

Поставьте 'x' в выполненных пунктах. Далее нажмите Shift+Enter.

- [x]  открыт файл;
- [x]  файл изучен;
- [x]  определены пропущенные значения;
- [x]  заполнены пропущенные значения;
- [x]  есть пояснение, какие пропущенные значения обнаружены;
- [x]  описаны возможные причины появления пропусков в данных;
- [x]  объяснено, по какому принципу заполнены пропуски;
- [x]  заменен вещественный тип данных на целочисленный;
- [x]  есть пояснение, какой метод используется для изменения типа данных и почему;
- [x]  удалены дубликаты;
- [x]  есть пояснение, какой метод используется для поиска и удаления дубликатов;
- [x]  описаны возможные причины появления дубликатов в данных;
- [x]  выделены леммы в значениях столбца с целями получения кредита;
- [x]  описан процесс лемматизации;
- [x]  данные категоризированы;
- [x]  есть объяснение принципа категоризации данных;
- [x]  есть ответ на вопрос: "Есть ли зависимость между наличием детей и возвратом кредита в срок?";
- [x]  есть ответ на вопрос: "Есть ли зависимость между семейным положением и возвратом кредита в срок?";
- [x]  есть ответ на вопрос: "Есть ли зависимость между уровнем дохода и возвратом кредита в срок?";
- [x]  есть ответ на вопрос: "Как разные цели кредита влияют на его возврат в срок?";
- [x]  в каждом этапе есть выводы;
- [x]  есть общий вывод.